In [1]:
"""This code will crate a master table for each tissue"""
#Set up
COLORS = {
    "Artery-Aorta":"salmon",
    "Artery-Tibial": "red",
    "Adipose-Subcutaneous": "darkorange",    
    "Adipose-Visceral":"orange",
    "Brain-Caudate":"lemonchiffon"   , 
    "Brain-Cerebellum":"yellow",
    "Cells-Transformedfibroblasts": "skyblue",
    "Esophagus-Mucosa": "sienna",
    "Esophagus-Muscularis":"burlywood",
    "Heart-LeftVentricle":"darkviolet",
    "Lung": "greenyellow",
    "Muscle-Skeletal": "mediumslateblue",
    "Nerve-Tibial":"gold",
    "Skin-NotSunExposed":"blue",
    "Skin-SunExposed":"cornflowerblue",
    "Thyroid":"green",
    "WholeBlood": "m"
        }

SHORTEN = {
    "Artery-Aorta":"Artery A."     ,
    "Artery-Tibial": "Artery T.",
    "Adipose-Subcutaneous": "Adipose S.",    
    "Adipose-Visceral":"Adipose V.",
    "Brain-Caudate":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "E. Mucosa",
    "Esophagus-Muscularis":"E Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed": "Skin Unexposed",
    "Skin-SunExposed":"Skin Leg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}

import math
import numpy as np
import os
import pandas as pd
import random
import sys
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

tissu = sorted(COLORS.keys())
CHR = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')
Motifs = pd.read_csv('/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed', sep='\t', header=None)
Motifs.columns = ['chrom','str.start','str.end','motif.len','motif']
path='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'

In [4]:
A=[]
B=[]
All_eSTRs=[]
for Tissue in tissu[:16]: 
    #command='mv '+path+Tissue+'/Master.table /storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/old_Master.table'
    #os.system(command)
    CAUSE=path+Tissue+'/HH/CAVIAR_analysis.table'
    SCORE=path+Tissue+'/HH/strs_scorechr'
    LR = path+Tissue+'/PQValues'
    ANOV = path+Tissue+'/HH/anova_wg.csv'
#LR
    lr = pd.read_csv(LR,sep='\t')
#Anova
    anova = pd.read_csv(ANOV, sep=',')
    anova = anova[['chrom','gene','str.start','anova_pval','estr_fdr','esnp_fdr','delta_bic','delta_aic']]
    #merge 1
    lr = pd.merge(lr, anova, on=['chrom','gene','str.start'], how='left')
#Causal 
    causal = pd.read_csv(CAUSE, sep='\t')
    causal['chrom']=causal['CHROM']
#Scores
    score_items=[]
    for i in CHR:
        C=pd.read_csv(SCORE+str(i), sep='\t', header=None)
        score_items.append(C)
    scores=score_items[0]
    for i in range(1, len(score_items)):
        scores=scores.append(score_items[i]) 
    scores.columns=['chrom', 'gene', 'str.id', 'caviar.score']
    CA = scores; CA.index = CA['gene']        
#Merge all in table
    Table = pd.merge(lr, CA, on=['chrom','gene','str.id'], how='left')
    Table.index = Table['gene']
    Master = pd.merge(Table,causal, on=['chrom','gene'], how='left')
    #print('With Caviar: ',Master.shape)
#Gene names
    genes=Gene_table.loc[Gene_table['gene'].isin(list(Master['gene']))]
    genes=genes[['gene','chrom','gene.name']] 
    Master=pd.merge(Master, genes, on=['chrom','gene'])
#motifs
    Master=pd.merge(Master, Motifs [['chrom','str.start','motif','str.end']], on=['chrom','str.start'])
#Additional columns
    Master['best.str'] = np.where(Master['qvalue']<=1, 1,0)
    Master['top.str.rank']=Master['str.rank']
    Master['top.snp.score']=Master['top_snp_score']
    Master['significant']= np.where(Master['qvalue']<=0.1, 1,0)
    Master['llsignificant'] = Master['llsignif'].astype(int)
    Master['top_str'] = Master['top_str'].astype(str)
    Master['top.variant'] = np.where(Master['top.str.score']>Master['top.snp.score'], Master['top_str'], Master['top_snp'])
    Master['best.str.start'] = Master['top_str'].astype(str).apply(lambda x: int(x.split('_')[1]) if 'STR' in x else '' )
#Summarize and savw
    Header = ['gene','chrom','str.start','str.id','best.str','best.str.start','top.variant','caviar.score','beta','beta.se','p.wald','qvalue','num.strs.in.top5','llqvalue','motif','top_str', 'top.str.score','top_snp', 'top.snp.score','top.str.rank','delta_bic','delta_aic','anova_pval','estr_fdr','esnp_fdr','llsignificant','significant', 'gene.name','str.end']
    Master = Master[Header].sort_values(['chrom','gene','str.start'])
    Master.to_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/Master.table', sep='\t',  index=None)
    #print('With gene name and motif : ',Master.shape, Tissue)
    
##   RUN The following
##   python ~/projects/GTEX_eSTRs/gtex-estrs/Scripts/STRs-vs-SNPs_effects/merge_causality_scores.py /storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/ WholeBlood,Cells-Transformedfibroblasts,Muscle-Skeletal,Lung,Artery-Tibial,Adipose-Subcutaneous,Esophagus-Mucosa causality > Merged_Best_causality.Table
##   To merge N select best by tissue
Master.transpose()

,0,5,9,12,14,19,24,28,1,6,...,87825,89139,89046,89048,89050,87273,87288,87305,87699,88178
gene,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000457.9,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,ENSG00000000460.12,...,ENSG00000268211.1,ENSG00000268926.1,ENSG00000268926.1,ENSG00000268926.1,ENSG00000268926.1,ENSG00000268996.1,ENSG00000268996.1,ENSG00000268996.1,ENSG00000269305.1,ENSG00000269305.1
chrom,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,...,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9,chr9
str.start,169735051,169784101,169895824,169937973,169614029,169667612,169701424,169701512,169735051,169784101,...,131063896,98477312,98543782,98626522,98631431,139894040,140014401,140029812,4765329,4871930
str.id,STR_169735051,STR_169784101,STR_169895824,STR_169937973,STR_169614029,STR_169667612,STR_169701424,STR_169701512,STR_169735051,STR_169784101,...,STR_131063896,STR_98477312,STR_98543782,STR_98626522,STR_98631431,STR_139894040,STR_140014401,STR_140029812,STR_4765329,STR_4871930
best.str,0,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,1,0,0,1
best.str.start,169784101,169784101,169784101,169784101,169667612,169667612,169667612,169667612,169667612,169667612,...,131063896,98631431,98631431,98631431,98631431,140014401,140014401,140014401,4871930,4871930
top.variant,SNP_169731382,SNP_169731382,SNP_169731382,SNP_169731382,SNP_169758289,SNP_169758289,SNP_169758289,SNP_169758289,SNP_169758289,SNP_169758289,...,SNP_131131399,SNP_98442830,SNP_98442830,SNP_98442830,SNP_98442830,SNP_140013296,SNP_140013296,SNP_140013296,SNP_4850431,SNP_4850431
caviar.score,0.00145263,0.00315061,0.00210653,0.00140446,3.51129e-11,4.90966e-10,2.95871e-11,2.95864e-11,4.2809e-10,3.02125e-11,...,0.0028016,0.000554015,0.00104182,0.00117953,0.00641936,0.000958418,0.00193232,0.00118026,0.000103388,0.000255539
beta,-0.0202151,0.0953801,-0.0679426,0.00454339,0.0421992,0.179192,-0.00105892,-0.000964771,0.172595,-0.0154379,...,-0.0442208,0.0123471,-0.0844552,-0.0925648,-0.161375,0.0845469,-0.120922,-0.0936167,-0.0302405,-0.106089
beta.se,0.0671019,0.0663627,0.0666609,0.0660812,0.0638307,0.0669401,0.0636284,0.064957,0.0661084,0.0666587,...,0.0648934,0.0668102,0.0658458,0.0660874,0.0643778,0.069593,0.0669253,0.0660808,0.064655,0.0667368
